In [ ]:
%env LD_LIBRARY_PATH /usr/local/cuda-10.0/lib64/

%load_ext autoreload
%autoreload 2

In [ ]:
import os
from keras import backend as K
import tensorflow as tf
import keras as ks
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras import layers
from keras import models
from keras.preprocessing import image
import numpy as np
from keras import optimizers
import matplotlib.pyplot as plt
from notebook_utils import *

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

if tf.test.gpu_device_name():
 print("Default GPU Device: {}".format(tf.test.gpu_device_name()))
else:
 print("GPU not working")

In [ ]:
base_dir = '/home/benoitpatra/code/twins-recognizer/data/sets'

TRAIN_DIR = os.path.join(base_dir, 'train')
VAL_DIR = os.path.join(base_dir, 'validation')
TEST_DIR = os.path.join(base_dir, 'test')

TARGET_SIZE = 160
BATCH_SIZE = 10
TRAIN_SET_SIZE=600
steps_per_epoch= TRAIN_SET_SIZE // BATCH_SIZE

In [ ]:
## Define TRAINING SET
train_datagen =  ImageDataGenerator(rescale=1./255, rotation_range=0.4,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              shear_range=0.2,
                              zoom_range=0.2,
                              horizontal_flip=True, fill_mode='nearest')


train_generator = train_datagen.flow_from_directory(TRAIN_DIR, 
                                                    target_size=(TARGET_SIZE, TARGET_SIZE), 
                                                    batch_size = BATCH_SIZE, 
                                                    class_mode='binary')

In [ ]:
## Define VALIDATION SET
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(VAL_DIR, 
                                                    target_size=(TARGET_SIZE, TARGET_SIZE), 
                                                    batch_size = BATCH_SIZE, 
                                                    class_mode='binary')

In [ ]:
img = next(train_generator)
print("Sample from generator")
print_image_from_gen(img[0][0,:,:,:])
print(img[1][0])

In [ ]:
def train_and_validate_model(model, epochs):
    t1 = time()
    model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])
    history = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=epochs, validation_data=validation_generator, validation_steps=50)
    t2 = time()
    elapsed = t2 - t1
    print('Elapsed time is %f seconds.' % elapsed)
    plot_history2(model.history)

In [ ]:
from adhoc_convnet import *

adhoc_convnet_model = adhoc_convnet(TARGET_SIZE)
adhoc_convnet_model.summary()

In [ ]:
from VGG16_base import *

vgg16_model = vgg16_base(TARGET_SIZE)
vgg16_model.summary()

In [ ]:
print("---loading---res_net---")
from inception_resnet_v1 import *
def res_net():
    model = InceptionResNetV1()

    #pre-trained weights https://drive.google.com/file/d/1971Xk5RwedbudGgTIrGAL4F7Aifu7id1/view?usp=sharing
    model.load_weights('facenet_weights.h5')
    
    new_model = models.Sequential()
    new_model.add(model)
    new_model.add(layers.Dense(256, activation='relu'))
    new_model.add(layers.Dropout(0.5))
    new_model.add(layers.Dense(64, activation='relu'))
    new_model.add(layers.Dropout(0.5))
    new_model.add(layers.Dense(16, activation='relu'))
    new_model.add(layers.Dense(1, activation='sigmoid'))

    return new_model

resnet_model = res_net()
resnet_model.summary()

In [ ]:
print("---EVALUATE RESNET MODEL--")
train_and_validate_model(resnet_model, 40)

In [ ]:
#resnet_model.save_weights("resnet_landj_20200419.h5")

In [ ]:
def get_test_generator():
    test_datagen = ImageDataGenerator(rescale=1./255)
    test_generator = test_datagen.flow_from_directory(TEST_DIR, 
                                                        target_size=(TARGET_SIZE, TARGET_SIZE), 
                                                        batch_size = 1, 
                                                        shuffle = False,
                                                        class_mode='binary')
    return test_generator

In [ ]:
batch = next(get_test_generator())
view_prediction(resnet_model, batch)

In [ ]:
TEST_SIZE=200
test_generator = get_test_generator()
result = resnet_model.evaluate_generator(test_generator, steps=TEST_SIZE)
print(result)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

target_names = ['J', 'L']
test_generator = get_test_generator()
Y_pred = resnet_model.predict_generator(test_generator, TEST_SIZE)
Y_pred = Y_pred.flatten()
y_pred_class = np.where(Y_pred > 0.5, 1, 0)

print('Confusion Matrix')

plot_confusion_matrix(confusion_matrix(test_generator.classes, y_pred_class),target_names)
print('Classification Report')

print(classification_report(test_generator.classes, y_pred_class, target_names=target_names))